In [301]:
import numpy as np
import random as rand
import plotly.graph_objects as go
from plotly.subplots import make_subplots


def rng(a=1.0, b=-1.0): return rand.uniform(a,b)


class Line2D:
    """ A randomized linear function \\
        f: R -> R ; f(x) = m*x + b
    """
    def __init__(self, m=rng(), b=rng()):
        self.m = m
        self.b = b

    def __call__(self, x):
        return x*self.m + self.b
# end


class Line3D:
    """ A randomized linear function \\
        f: R2 -> R ; f(x,y) = mx*x + my*y + b
    """
    def __init__(self, mx=rng(), my=rng(), b=rng()):
        self.mx = mx
        self.my = my
        self.b = b

    def __call__(self, x, y):
        return x*self.mx + y*self.my + self.b
# end


def reg(xx, yy) -> 'function':
    """ Linear regression using analytic minimized loss \\
        Returns a regression line in Rm as a lambda \\
        f: R(nxm) -> g: Rm -> R
    """
    X = np.column_stack((np.ones((len(xx),1)), xx))
    A = np.linalg.inv(np.matmul(X.transpose(), X))
    b = X.transpose().dot(yy)
    w = A.dot(b)
    
    return lambda x: np.column_stack((np.ones((len(x),1)), x)).dot(w)
# end

In [22]:
f = Line2D(2, 50)
xx = np.linspace(-10, 10, 50)
yy = [f(x) + rng(-5, 5) for x in xx]

reg_line = reg(xx, yy)(xx)

trace1 = go.Scatter(
    x=xx, y=reg_line,
    line=dict(width=5),
    name='best fit',
    mode='lines'
)

trace2 = go.Scatter(
    x=xx, y=yy,
    name='data',
    mode='markers'
)

layout = go.Layout(width=900, height=900)
fig = go.Figure(data=[trace1, trace2], layout=layout)
fig.show()

In [19]:
xy = np.linspace(-1, 1, 40)
fxy = Line3D()

noisy_x = [x + rng(-0.3, 0.3) for x in xy]
noisy_y = [y + rng(-0.3, 0.3) for y in xy]
noisy_z = [fxy(x,y) + rng(-0.3, 0.3) for (x,y) in zip(xy,xy)]

reg_line = reg([*zip(noisy_x, noisy_y)], noisy_z)(np.column_stack((xy,xy)))

trace1 = go.Scatter3d(
    x=xy, y=xy, z=reg_line,
    line=dict(width=10),
    name='best fit',
    mode='lines'
)

trace2 = go.Scatter3d(
    x=noisy_x, y=noisy_y, z=noisy_z,
    name='data',
    mode='markers'
)

layout = go.Layout(width=900, height=900)
fig = go.Figure(data=[trace1,trace2], layout=layout)
fig.show()

## Imports

In [278]:
import numpy as np
import random as rand
import plotly.graph_objects as go
from plotly.subplots import make_subplots


# A helper function to reduce the verbosity of the code to follow
def rng(a=1.0, b=-1.0): return rand.uniform(a,b)


class Line2D:
    """ A linear function with optional randomized slope and intercept \\
        f: R -> R ; f(x) = m*x + b
    """
    def __init__(self, m=rng(), b=rng()):
        self.m = m
        self.b = b

    def __call__(self, x):
        return x*self.m + self.b
# end


def reg(xx, yy) -> 'function':
    """ Linear regression using analytic minimized loss \\
        Returns a regression line in Rm as a lambda \\
        f: R(nxm) -> g: Rm -> R
    """
    X = np.column_stack((np.ones((len(xx),1)), xx))
    A = np.linalg.inv(np.matmul(X.transpose(), X))
    b = X.transpose().dot(yy)
    w = A.dot(b)
    
    return lambda x: np.column_stack((np.ones((len(x),1)), x)).dot(w)
# end


def sigmoid(x):
    return (1+np.exp(np.log(99) - abs(x)))**(-1)


def rate_limiter(x, k=0.9):
    return 1.0 - k*np.exp(-10*x**2)


def grad(xx, yy, tol=1e-5):
    w_rate = 1e-4
    b_rate = 1e-2

    max_iter = 1000    
    xy_zip = zip(xx,yy)

    b_path = [rng(2, 98)]
    w_path = [rng(-8, 8)]
    ones = np.ones(len(xx))
    num_iter = 0

    for n in range(max_iter):
        num_iter = n

        w0, b0 = w_path[-1], b_path[-1]

        w_err = xx.dot(w0*xx + b0 - yy)
        b_err = ones.dot(w0*xx + b0 - yy)
        
        if abs(w_err) < tol:
            w_path.append(w0)
        else:
            w_rate = (5e-4)*rate_limiter(abs(w_err))
            w1 = w0 - w_rate*w_err
            w_path.append(w1)

        if abs(b_err) < tol:
            b_path.append(b0)
        else:
            b_rate = (1e-2)*rate_limiter(100*abs(b_err), k=0.5)
            b1 = b0 - b_rate*b_err
            b_path.append(b1)

        if (abs(w_err) < tol) & (abs(b_err) < tol):
            break

    return b_path, w_path, num_iter
# end

## Generate data, record error

In [163]:
# Create a slope-intercept function 'object' with slope 2, and intercept 50
f = Line2D(2, 50)
xx = np.linspace(-10, 10, 50)
yy = [f(x) + rng(-5, 5) for x in xx]

# Bias and weight vectors
bb = np.arange(0,101,1)
ww = np.arange(-10,11,1)

# Error as a function of bias and weight
Z = np.zeros((len(bb), len(ww)))

for i, b in enumerate(bb):
    for j, w in enumerate(ww):
        for n, (x, y) in enumerate(zip(xx,yy)):
            Z[(i,j)] += (w*x + b - y)**2
        # end
    # end
# end

# scale the output
Z = Z/1000

## Plot error "landscape"

In [283]:
b_path, w_path, num_iter = grad(xx, yy, 1e-4)

loss_title = f'Gradient descent path on error surface, n={num_iter} iterations'

loss_trace = go.Contour(
    x=ww, y=bb, z=Z,
    contours=dict(
        showlabels=True,
        labelfont=dict(
            size=12,
            color='gray'
        ),
        start=10, size=50, end=310
    ),
    colorscale=[
        [0.0, 'mediumturquoise'],
        [0.6, 'gold'],
        [1.0, 'lightsalmon']
    ],
    showscale=False,
    line_width=0
)

path_trace = go.Scatter(
    x=w_path, y=b_path,
    name='gradient descent path',
    mode='lines+markers',
    marker_size=12,
    marker_color='white',
    line_width=5,
    line_color='lightgray',
)

true_min_trace = go.Scatter(
    x=[2], y=[50],
    name='true minimum (w=2, b=50)',
    mode='markers',
    marker_size=15,
    marker_color='lightsalmon',
    marker_symbol='x'
)

fig = go.Figure(
    data=[loss_trace, path_trace, true_min_trace],
    layout=go.Layout(
        width=950, height=950,
        title=loss_title,
        xaxis_title='weight values',
        yaxis_title='bias values',
        legend_x=0, legend_y=1,
        legend_bgcolor='rgba(0,0,0,0.3)',
        legend_font_color='white'
    )
)

fig.show()

In [236]:
b = b_path[-1]
w = w_path[-1]

def reg_grad(x): return w*x + b

predicted_trace = go.Scatter(
    x=xx, y=reg_grad(xx),
    line=dict(width=5),
    name='predicted',
    mode='markers',
    marker_size=10,
    marker_color='mediumturquoise',
    marker_symbol='x'
)

target_trace = go.Scatter(
    x=xx, y=f(xx),
    name='target',
    mode='markers',
    marker_size=15,
    marker_color='gold'
)

error_trace = go.Scatter(
    x=xx, y=abs(f(xx)-reg_grad(xx)),
    name='error',
    mode='lines',
    line_width=5,
    line_color='lightsalmon'
)

fig = make_subplots(
    rows=2, cols=1,
    row_heights=[0.8, 0.2],
    subplot_titles=('<b>Predicted values vs real values</b>', ''),
    vertical_spacing=0.025, shared_xaxes=True
)
fig.add_trace(target_trace, row=1, col=1)
fig.add_trace(predicted_trace, row=1, col=1)
fig.add_trace(error_trace, row=2, col=1)

# Update xaxis properties
# fig.update_xaxes(title_text='<b>X</b>', row=1, col=1)
fig.update_xaxes(title_text='<b>X</b>', row=2, col=1)

# Update yaxis properties
fig.update_yaxes(title_text='<b>Y</b>', row=1, col=1)
fig.update_yaxes(title_text='<b>ABSOLUTE ERROR</b>', titlefont_color='#ff7f0e', row=2, col=1)

fig.update_layout(height=950, width=950, plot_bgcolor='lightgray')

fig.show()